 #  A Whale off the Port(folio)

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
import seaborn as sns
%matplotlib inline

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:
1. whale_returns.csv
2. algo_returns.csv
3. sp500_history.csv

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [ ]:
# Reading whale returns
whale_returns_csv = Path("Resources/whale_returns.csv")
whale_returns = pd.read_csv(whale_returns_csv, index_col='Date', infer_datetime_format=True, parse_dates=True)
whale_returns.head()

In [ ]:
# Count nulls
whale_returns.isna().sum()

In [ ]:
# Drop nulls
whale_returns = whale_returns.dropna()
whale_returns.isna().sum()


## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [ ]:
# Reading algorithmic returns
algo_returns_csv = Path("Resources/algo_returns.csv")
algo_returns = pd.read_csv(algo_returns_csv, index_col='Date', infer_datetime_format=True, parse_dates=True)
algo_returns.head()

In [ ]:
# Count nulls
algo_returns.isna().sum()

In [ ]:
# Drop nulls
algo_returns = algo_returns.dropna()
algo_returns.isna().sum()


## S&P 500 Returns

Read the S&P500 Historic Closing Prices and create a new daily returns DataFrame from the data. 

In [ ]:
# Reading S&P 500 Closing Prices
sp500_history_csv = Path("Resources/sp500_history.csv")
sp500_history = pd.read_csv(sp500_history_csv, index_col='Date', infer_datetime_format=True, parse_dates=True)
sp500_history.sort_index(inplace=True)
sp500_history.head()


In [ ]:
# Check Data Types
sp500_history.dtypes

In [ ]:
# Fix Data Types
sp500_history["Close"] = sp500_history["Close"].str.replace("$","")
sp500_history["Close"] = sp500_history["Close"].astype("float")
sp500_history.dtypes

In [ ]:
# Calculate Daily Returns
sp500_returns = sp500_history.pct_change()
sp500_returns.head()

In [ ]:
# Drop nulls
sp500_returns.dropna().head()


In [ ]:
# Rename Column
sp500_returns.rename(columns={"Close": "S&P 500"}, inplace=True)
sp500_returns.head()

## Combine Whale, Algorithmic, and S&P 500 Returns

In [ ]:
# Concatenate all DataFrames into a single DataFrame
combined_returns = pd.concat([whale_returns, algo_returns, sp500_returns], axis="columns", join="inner")
combined_returns.head()

---

# Portfolio Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance

Calculate and Plot the daily returns and cumulative returns. Does any portfolio outperform the S&P 500? 

In [ ]:
# Plot daily returns
combined_returns.plot(title="Daily Returns", figsize=(20,10))

In [ ]:
# Plot cumulative returns
cumulative_returns = (1 + combined_returns).cumprod()
cumulative_returns.plot(title="Cumulative Returns", figsize=(20,10))

### *Algo 1 out performed the S&P 500.


## Risk

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

In [ ]:
# Box plot to visually show risk
combined_returns.boxplot(figsize=(20,10)).set_title("Portfolio Risk")


In [ ]:
# Daily Standard Deviations
# Calculate the standard deviation for each portfolio. 
# Which portfolios are riskier than the S&P 500?
combined_daily_std = combined_returns.std()
combined_daily_std

### *Tiger Global Management LLC and Berkshire Hathaway Inc have greater risk compared to the S&P 500.

In [ ]:
# Determine which portfolios are riskier than the S&P 500
index_num = 0
for portfolio in combined_daily_std:
    if portfolio > combined_daily_std["S&P 500"]:
        true_false = "True"
        print(f"{combined_daily_std.index[index_num]:<30}{true_false:>6}")
    else:
        true_false = "False"
        print(f"{combined_daily_std.index[index_num]:<30}{true_false:>6}")
    index_num += 1

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annualized_combined_std = combined_daily_std * np.sqrt(252)
annualized_combined_std

---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Plot the rolling standard deviation of the various portfolios along with the rolling standard deviation of the S&P 500 (consider a 21 day window). Does the risk increase for each of the portfolios at the same time risk increases in the S&P?
2. Construct a correlation table for the algorithmic, whale, and S&P 500 returns. Which returns most closely mimic the S&P?
3. Choose one portfolio and plot a rolling beta between that portfolio's returns and S&P 500 returns. Does the portfolio seem sensitive to movements in the S&P 500?
4. An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the ewm with a 21 day half-life.

In [ ]:
# Calculate and plot the rolling standard deviation for
# the S&P 500 and whale portfolios using a 21 trading day window
combined_returns.rolling(window=21).std().plot(figsize=(20,10), title="21 Day Rolling Standard Deviation")

In [ ]:
# Construct a correlation table
correlation = combined_returns.corr()
sns.heatmap(correlation, vmin=0, vmax=1, cmap="YlGn", annot=True).set_title("Correlation")

In [ ]:
# Calculate Beta for a single portfolio compared to the total market (S&P 500)
# (Your graph may differ, dependent upon which portfolio you are comparing)
rolling_covariance = combined_returns['BERKSHIRE HATHAWAY INC'].rolling(window=21).cov(combined_returns['S&P 500'])
rolling_variance = combined_returns['S&P 500'].rolling(window=21).var()
rolling_beta = rolling_covariance / rolling_variance
rolling_beta.plot(figsize=(20,10), title="Berkshire Hathaway Inc Beta")

In [ ]:
# Calculate a rolling window using the exponentially weighted moving average.
expo_rolling_window = combined_returns.ewm(halflife=21).mean()
expo_rolling_window.plot(figsize=(20,10), title="Exponential Weighted Average")

---

## Sharpe Ratios
In reality, investment managers and their institutional investors look at the ratio of return-to-risk, and not just returns alone. (After all, if you could invest in one of two portfolios, each offered the same 10% return, yet one offered lower risk, you'd take that one, right?)

1. Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot.
2. Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

In [ ]:
# Calculate annualized Sharpe Ratios
sharpe_ratios = (combined_returns.mean() * 252) / (combined_returns.std() * np.sqrt(252))
sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
sharpe_ratios.plot(kind = 'bar', title = 'Sharpe Ratios')

On the basis of this performance metric, do our algo strategies outperform both 'the market' and the whales? Type your answer here:

### *Based on this performance metric, Algo 1 outperforms both 'the market' and the whales, while Algo 2 does not outperform them.


---

# Portfolio Returns

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Visit [Google Sheets](https://docs.google.com/spreadsheets/) and use the in-built Google Finance function to choose 3-5 stocks for your own portfolio.
2. Download the data as CSV files and calculate the portfolio returns.
3. Calculate the returns for each stock.
4. Using those returns, calculate the weighted returns for your entire portfolio assuming an equal number of shares for each stock.
5. Add your portfolio returns to the DataFrame with the other portfolios and rerun the analysis. How does your portfolio fair?


## Your analysis should include the following:

- Using all portfolios:
 - The annualized standard deviation (252 trading days) for all portfolios.
 - The plotted rolling standard deviation using a 21 trading day window for all portfolios.
 - The calculated annualized Sharpe Ratios and the accompanying bar plot visualization.
 - A correlation table.
- Using your custom portfolio and one other of your choosing:
 - The plotted beta. 

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [ ]:
# Read the first stock
chrw_csv = Path("Resources/CHRW.csv")
chrw_data = pd.read_csv(chrw_csv, index_col='Date', parse_dates=True, infer_datetime_format=True)
chrw_data.head()

In [ ]:
# Read the second stock
xom_csv = Path("Resources/XOM.csv")
xom_data = pd.read_csv(xom_csv, index_col='Date', parse_dates=True, infer_datetime_format=True)
xom_data.head()

In [ ]:
# Read the third stock
zts_csv = Path("Resources/zts.csv")
zts_data = pd.read_csv(zts_csv, index_col='Date', parse_dates=True, infer_datetime_format=True)
zts_data.head()

In [ ]:
# Concatenate all stocks into a single DataFrame
combined_data = pd.concat([chrw_data, xom_data, zts_data], axis="columns", join="inner")
combined_data.head()

In [ ]:
# Pivot so that each column of prices represents a unique symbol
combined_data.columns = ['CHRW', 'XOM', 'ZTS']
combined_data.head()

In [ ]:
# Drop Nulls
daily_returns = combined_data.pct_change()
daily_returns = daily_returns.dropna()
daily_returns.head()

## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [ ]:
# Calculate weighted portfolio returns
weights = [1/3, 1/3, 1/3]
portfolio_weighted_returns = daily_returns.dot(weights)
portfolio_weighted_returns.head()

## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [ ]:
# Add your "Custom" portfolio to the larger dataframe of fund returns
all_portfolios_returns = pd.concat([combined_returns, portfolio_weighted_returns], axis="columns", join="inner")
all_portfolios_returns.rename(columns={0: "Custom"}, inplace=True)
all_portfolios_returns.head()

In [ ]:
# Only compare dates where return data exists for all the stocks (drop NaNs)
all_portfolios_returns.isna().sum()

## Re-run the performance and risk analysis with your portfolio to see how it compares to the others

In [ ]:
# Risk
portfolios_risk = all_portfolios_returns.std() * np.sqrt(252)
portfolios_risk

In [ ]:
# Rolling
all_portfolios_returns.rolling(window=21).std().plot(figsize=(20,10), title="21 Day Rolling Standard Deviation")

In [ ]:
# Annualized Sharpe Ratios
all_annualized_sharpe_ratios = (all_portfolios_returns.mean() * 252) / (portfolios_risk)
all_annualized_sharpe_ratios

In [ ]:
# Visualize the sharpe ratios as a bar plot
all_annualized_sharpe_ratios.plot(kind='bar', figsize=(15,5), title='Annualized Sharpe Ratios')

In [ ]:
# Create a correlation analysis
all_correlations = all_portfolios_returns.corr()
sns.heatmap(all_correlations, vmin=0, vmax=0.9, cmap="YlGn", annot=True).set_title("Portfolios Correlations")

In [ ]:
# Beta
custom_rolling_covariance = all_portfolios_returns['Custom'].rolling(window=21).cov(all_portfolios_returns['S&P 500'])
custom_rolling_variance = all_portfolios_returns['S&P 500'].rolling(window=21).var()
custom_rolling_beta = custom_rolling_covariance / custom_rolling_variance
custom_rolling_beta.plot(figsize=(20,10), title="Beta for Custom Portfolio")